In [1]:
import ollama
from pinecone import Pinecone, ServerlessSpec
import os
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

C:\Users\Tomas\AppData\Roaming\Python\Python310\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# 파일 위치 지정
keys_file_path = os.path.join('data', 'api_keys.txt')

# 파일에서 API 키를 로드하는 함수
def load_api_keys(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keys = json.load(file)
    return keys

# API 키 사용
api_keys = load_api_keys(keys_file_path)
openAI_keys = api_keys['openAI_keys']
pinecone_key1 = api_keys['pinecone_key1']
pinecone_key2 = api_keys['pinecone_key2']

In [3]:
# Pinecone 연결 및 index 설정
pc = Pinecone(api_key=pinecone_key2)

# key1 => minilm / key2 => mpnet
index = pc.Index('mpnet')

index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.41754,
 'namespaces': {'': {'vector_count': 41754}},
 'total_vector_count': 41754}

In [4]:
file_path = os.path.join(os.getcwd(), "data", "processed_data.txt")

# 문장을 저장할 배열 초기화
sentences = []

# 파일 열기
with open(file_path, "r", encoding="utf-8") as file:
    # 파일의 각 줄을 읽어와 배열에 추가
    for line in file:
        sentences.append(line.strip())

In [8]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Assume sentences is a list of input sentences
embedded_sentences = []

# Encode each sentence and store the embeddings
for sentence in tqdm(sentences):
    embedded_sentences.append(model.encode(sentence))

print(embedded_sentences[0])

100%|████████████████████████████████████████████████████████████████████████████████| 186/186 [00:23<00:00,  7.80it/s]


[array([ 2.08201744e-02, -1.07037596e-01,  2.14415211e-02, -2.68955510e-02,
        1.31547293e-02, -1.39538571e-02, -5.96780330e-02, -1.33626806e-02,
        5.38083836e-02,  1.43554788e-02, -1.85819734e-02,  3.86043340e-02,
       -1.62289664e-02, -2.65340004e-02,  2.53813956e-02,  3.68113117e-03,
        5.00071533e-02, -1.22267585e-02,  5.08567207e-02,  6.88743778e-04,
       -6.02892153e-02, -2.90247258e-02,  2.58078594e-02,  1.80703532e-02,
       -2.36614328e-02, -3.33530386e-03, -9.54449642e-03, -1.60108935e-02,
       -4.21916135e-02, -5.46915457e-02, -2.66851559e-02,  8.36803298e-03,
       -2.46263295e-02,  4.35893610e-02,  1.58004116e-06,  1.32559072e-02,
        6.32733926e-02,  2.15729256e-03, -2.68571861e-02, -6.21924326e-02,
        9.08079892e-02,  1.58581957e-02, -1.68577712e-02, -2.92072102e-04,
        2.28485502e-02, -1.97100956e-02,  3.42469960e-02,  2.12272946e-02,
        9.36810579e-03, -6.27371818e-02,  1.00637553e-03, -3.88962217e-02,
       -6.14600908e-03, 

In [14]:
# Initialize an empty list to collect all retrieved assertions
all_assertions = []

# Loop through each encoded sentence and query the index for matches
for i, encoded_sentence in enumerate(tqdm(embedded_sentences)):
    # Query the index using the current encoded sentence
    matches = index.query(
        vector=encoded_sentence.tolist(),
        top_k=5,  # Adjust the number of matches to retrieve as needed
        include_values=False,
        include_metadata=True,
    )
    
    # Extract the assertions metadata from each match
    assertions_list = [match['metadata']['assertion'].strip() for match in matches['matches']]

    # Collect all results in the overall list
    all_assertions.append(' '.join(assertions_list))

# If you want to store or process the combined assertions list
print(all_assertions)

100%|████████████████████████████████████████████████████████████████████████████████| 186/186 [00:50<00:00,  3.65it/s]

['Koreans use chopsticks and spoons to eat their meals. Korean chopsticks are typically made of metal, are flat, and have blunt or sharp ends. Korean food is unique and exciting, with a variety of flavors, textures, and aromas. Korean food is no stranger to many cultures and is known for its unique and delicious dishes. Koreans usually eat rice with a spoon, which is different from other Asian countries.', 'Turkish food is a diverse and delicious cuisine that is becoming more popular all over the world. Turkish cuisine uses a variety of spices to enhance the flavor of their dishes. Turkish tea is a big part of Turkish cuisine and is served in many different ways. Turkey is known for its spices, which are used in many different ways in Turkish cuisine. Turkish coffee is a delicious and unique coffee drink that is enjoyed by many.', 'The cuisine of Egypt has a long and rich history, with many dishes dating back to ancient times. The cuisine of Egypt is a very rich cuisine that has many u

In [ ]:
matches = index.query(
    vector=model.encode(embedded_sentences[0]),
    top_k=5,
    include_values=False,
    include_metadata=True,
)

In [ ]:
response = ollama.chat(model='llama2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [ ]:
# 찾아온 벡터들의 토큰화된 metadata들 다시 이어 붙임
assertions_list = [match['metadata']['assertion'].strip() for match in matches['matches']]

print(assertions_list)